# Data Extraction

This notebook is for extracting the data from the source.

Here, we will be extracting fresh data from the source or updating the existing data in our project.

The source of the data is from the official Danish (Denmark) energy system which is a free and open data portal website.

Note: This notebook is for experimenting purpose, complete code is in the 'src' directory.

In [1]:
import os
import datetime
import requests
import pandas as pd
from pathlib import Path
from json import dump, JSONDecodeError

In [2]:
# Changing the working directory to project directory
NOTEBOOK_DIR_PATH = Path('D:/projects/energy-consumption-forecasting/notebooks')

if Path(os.getcwd()) == NOTEBOOK_DIR_PATH:
    os.chdir(path='..')

os.getcwd()

'd:\\projects\\energy-consumption-forecasting'

In [3]:
# Data path for storing all the data
DATA_DIR_PATH = Path('data/')
DATA_DIR_PATH

WindowsPath('data')

Getting the data is quite simple, the data portal contains an API guide which has all the instructions that are needed for extracting the data.

Note: It is been mentioned that even though the data is free to download there are various limitations and restriction.\
For complete understanding kindly go through the API guide: https://www.energidataservice.dk/guides/api-guides

In [4]:
# A function for formatting the datetime that matches the requirement for API
def get_extraction_datetime(start_date_time: datetime.datetime,
                            end_date_time: datetime.datetime,):
    
    # Checking the datatype for start and end date are datetime.datetime
    if not(isinstance(start_date_time, datetime.datetime) and 
           isinstance(end_date_time, datetime.datetime)):
        raise TypeError('Invalid type of arguments for start and end date, '
                        'must be a datetime.datetime data type')
    
    # Checking if end date is greater than start date
    assert start_date_time < end_date_time, 'End date needs to be greater than the start date' 
    
    # Converting the date format for API query and  
    # increasing the end date by 1 day as per the API guide (off-by-one error)
    start_date_time = start_date_time.strftime('%Y-%m-%dT%H:%M')
    end_date_time = (end_date_time 
                     + datetime.timedelta(days=1)).strftime('%Y-%m-%dT%H:%M')

    return start_date_time, end_date_time

In [5]:
start, end = get_extraction_datetime(start_date_time=datetime.datetime(2021, 1, 1),
                                     end_date_time=datetime.datetime(2021, 1, 5))
print(f'Star Date: {start} and End Date: {end}')

Star Date: 2021-01-01T00:00 and End Date: 2021-01-06T00:00


In [6]:
def extract_dataset_from_api(start_date_time: datetime.datetime,
                             end_date_time: datetime.datetime,
                             sort_data_asc: bool = True,
                             dataset_name: str = 'ConsumptionIndustry',
                             base_url: str = 'https://api.energidataservice.dk/dataset/',
                             meta_url: str = 'https://api.energidataservice.dk/meta/dataset/',
                             save_dataset_metadata: bool = True):
    
    data_url = f'{base_url}{dataset_name}?'
    meta_url = f'{meta_url}{dataset_name}?'
    sort = 'HourUTC' if sort_data_asc else 'HourUTC%20DESC'

    # Formatting the dates for the API parameters
    start, end = get_extraction_datetime(start_date_time=start_date_time,
                                         end_date_time=end_date_time)
    
    # Creating the parameters for the API request
    params = {'offset': 0,
              'start': start,
              'end': end,
              'sort': sort,}
    
    # Calling the API requests for dataset and metadata
    with requests.Session() as session:
        print(f'Sending API get request to: {data_url} and {meta_url} with parameters: {params}.\n')
        data_response = session.get(url=data_url, params=params)
        meta_response = session.get(url=meta_url)
        print(f'Connection to the dataset API is done and response received with status code: {data_response.status_code}.')
        print(f'Connection to the metadata API is done and response received with status code: {meta_response.status_code}.\n')

    try :
        json_data = data_response.json()
        json_meta = meta_response.json()
    except JSONDecodeError:
        print(f'Error status code for Data: {data_response.status_code} and meta: {meta_response.status_code} while decoding the response into JSON format, recheck the get request method.')
        return None
    
    # Getting the dataset from the JSON data and converting into dataframe
    json_data = json_data.get('records')
    dataset_df = pd.DataFrame.from_records(json_data)

    if save_dataset_metadata:
        
        data_dir = Path('./data')
        if not os.path.isdir(data_dir):
            os.makedirs(data_dir)

        start = start.replace(':', '-')
        end = end.replace(':', '-')
        data_filepath = data_dir / f'{dataset_name}_{start}_{end}.csv'
        meta_filepath = data_dir / f'{dataset_name}_metadata.json'
        print(f'Saving the dataset "{data_filepath.name}" in directory: "{data_dir.absolute()}".')
        print(f'Saving the metadata "{meta_filepath.name}" in directory: "{data_dir.absolute()}".\n')

        # Saving the dataset as a csv file and meta data as JSON file in data directory
        dataset_df.to_csv(path_or_buf=data_filepath,
                          index=False)
        with open(file=meta_filepath, mode='w') as file:
            dump(obj=json_meta, fp=file)
        print(f'Dataset has been saved in csv file "{data_filepath.name}".')
        print(f'Metadata has been saved in json file "{meta_filepath.name}".\n')

        return dataset_df, json_meta, data_filepath, meta_filepath

    return dataset_df, json_meta

In [7]:
dataset_df, json_meta, data_filepath, meta_filepath = extract_dataset_from_api(start_date_time=datetime.datetime(2021, 1, 1),
                                                                               end_date_time=datetime.datetime(2021, 1, 2))

Sending API get request to: https://api.energidataservice.dk/dataset/ConsumptionIndustry? and https://api.energidataservice.dk/meta/dataset/ConsumptionIndustry? with parameters: {'offset': 0, 'start': '2021-01-01T00:00', 'end': '2021-01-03T00:00', 'sort': 'HourUTC'}.

Connection to the dataset API is done and response received with status code: 200.
Connection to the metadata API is done and response received with status code: 200.

Saving the dataset "ConsumptionIndustry_2021-01-01T00-00_2021-01-03T00-00.csv" in directory: "d:\projects\energy-consumption-forecasting\data".
Saving the metadata "ConsumptionIndustry_metadata.json" in directory: "d:\projects\energy-consumption-forecasting\data".

Dataset has been saved in csv file "ConsumptionIndustry_2021-01-01T00-00_2021-01-03T00-00.csv".
Metadata has been saved in json file "ConsumptionIndustry_metadata.json".



In [8]:
dataset_df

,HourUTC,HourDK,MunicipalityNo,Branche,ConsumptionkWh
0,2020-12-31T23:00:00,2021-01-01T00:00:00,860,Privat,19864.474
1,2020-12-31T23:00:00,2021-01-01T00:00:00,860,Offentligt,2237.222
2,2020-12-31T23:00:00,2021-01-01T00:00:00,860,Erhverv,23567.818
3,2020-12-31T23:00:00,2021-01-01T00:00:00,851,Privat,37842.849
4,2020-12-31T23:00:00,2021-01-01T00:00:00,851,Offentligt,10517.319
...,...,...,...,...,...
14107,2021-01-02T22:00:00,2021-01-02T23:00:00,147,Offentligt,3753.202
14108,2021-01-02T22:00:00,2021-01-02T23:00:00,147,Erhverv,13016.514
14109,2021-01-02T22:00:00,2021-01-02T23:00:00,101,Privat,67934.111
14110,2021-01-02T22:00:00,2021-01-02T23:00:00,101,Offentligt,28250.310


In [9]:
json_meta

{'datasetId': 18,
 'datasetName': 'ConsumptionIndustry',
 'title': 'Consumption per Industry, Public and Private, Municipality and Hour',
 'description': 'Hourly consumption of industries, public sector and private units by municipality code.',
 'organizationName': 'tso-electricity',
 'updateFrequency': 'P1D',
 'comment': "* Categories are created based on Statistics Denmark's DK10 Industry Codes:\n* \\- The category 9 ('Offentlig, undervisning og sundhed') is labeled 'Offentligt' (public).\n* \\- Units connected to a CVR number in the other of Statistics Denmark's categories, including category 11 ('Uoplyst aktivitet') are labeled 'Erhverv' (industry).\n* \\- Other units (without CVR numbers) are labeled as 'Privat' (private).\n\nUpdated gradually since 2021.  ",
 'filterColumn': 'HourUTC',
 'author': 'Energinet',
 'resolution': '1 hour (PT1H)',
 'columns': [{'dbColumn': 'HourUTC',
   'dataType': 'datetime',
   'sortOrder': 1,
   'primaryKeyIndex': 1,
   'displayName': 'Hour UTC',
   

In [10]:
data_filepath, meta_filepath

(WindowsPath('data/ConsumptionIndustry_2021-01-01T00-00_2021-01-03T00-00.csv'),
 WindowsPath('data/ConsumptionIndustry_metadata.json'))